In [1]:
from pydantic_wrap import SimplePageFuncPydanticWithOutput, pydantic_model_from_type

In [2]:
def ff(x:int)->int:
    return x

mytype = ff.__annotations__["return"]

mod = pydantic_model_from_type(mytype)

In [3]:
mod.__fields__

{'result': ModelField(name='result', type=int, required=True)}

# flat dispatch

In [6]:
from streamlitfront import dispatch_funcs
from front.scrap.pydantic_wrap import SimplePageFuncPydanticWithOutput
from typing import Optional
from streamlit_pydantic.types import FileContent
from pydantic import Field, BaseModel
from dol import Store




In [11]:
def func(salary: float, n_months=12):
    return salary * n_months

salary_store = {
    'sylvain': 10000,
    'christian': 2000,
    'thor': 50000
}

def store_wrapped_func(salary: str, n_months=12):
    salary = salary_store[salary]
    return func(salary, n_months)

assert store_wrapped_func('sylvain', 6) == 60000

In [17]:
from i2.wrapper import wrap

def ingress_salary(salary_str:str, n_months:int):
    return (salary_store[salary_str], n_months), dict()

wrapped = wrap(func, ingress = ingress_salary)
assert wrapped('sylvain', 12)==120000

In [ ]:
"""A simple app to demo the use of Mappings to handle complex type"""
from typing import Mapping

ComplexType = float  # just pretend it's complex!


def func(salary: ComplexType, n_months: int = 12):
    return salary * n_months


SalaryKey = str  # or some type that will resolve in store-fed key selector
SalaryMapping = Mapping[SalaryKey, ComplexType]

salary_store: SalaryMapping
salary_store = {"sylvain": 10000, "christian": 2000, "thor": 50000}


def store_wrapped_func(salary: SalaryKey, n_months: int = 12):
    salary = salary_store[salary]
    return func(salary, n_months)


assert store_wrapped_func("sylvain", 6) == 60000


if __name__ == "__main__":
    from streamlitfront.base import dispatch_funcs

    app = dispatch_funcs([func, store_wrapped_func])
    app()

In [20]:
"""A simple app to demo the use of Mappings to handle complex type"""
from typing import Mapping
from enum import Enum

from i2.wrapper import wrap



ComplexType = float  # just pretend it's complex!


def func(salary: ComplexType, n_months: int = 12):
    return salary * n_months


SalaryKey = str  # or some type that will resolve in store-fed key selector
SalaryMapping = Mapping[SalaryKey, ComplexType]

salary_store: SalaryMapping
salary_store = {"sylvain": 10000, "christian": 2000, "thor": 50000}

def choose_key(choice):
    pass

def ingress_salary(salary_str:SalaryKey, n_months:int):
    return (salary_store[salary_str], n_months), dict()

store_wrapped_func = wrap(func, ingress = ingress_salary)


assert store_wrapped_func("sylvain", 6) == 60000

class SelectionValue(str, Enum):
    FOO = "foo"
    BAR = "bar"


class ExampleModel(BaseModel):
    single_selection: SelectionValue = Field(
        ..., description="Only select a single item from a set."
    )

In [21]:
class SelectionValue(str, Enum):
    FOO = "foo"
    BAR = "bar"


class ExampleModel(BaseModel):
    single_selection: SelectionValue = Field(
        ..., description="Only select a single item from a set."
    )

In [30]:
from dataclasses import dataclass, field, make_dataclass
from typing import List

store = {'sly':20000, 'chris':4000}
def mk_choices():
    return list(store.keys())

@dataclass
class C:
    mylist: List[str] = field(default_factory=mk_choices)

c = C()
c.mylist += [1, 2, 3]

In [31]:
DFLT_CHOICES = ['sly','chris']
C = Enum("Choices", {'sly':'sly', 'chris':'chris'})

In [32]:
C

<enum 'Choices'>

# Scrap

In [13]:
"""
Same as take_04_model_run, but where the dispatch is not as manual.
"""

# This is what we want our "dispatchable" wrapper to look like

# There should be real physical stores for those types (FVs, FittedModel) that need them
from typing import Any, Mapping, Tuple

import numpy as np
from sklearn.preprocessing import MinMaxScaler

FVs = Any
FittedModel = Any

# ---------------------------------------------------------------------------------------
# The function(ality) we want to dispatch:
def apply_model(fitted_model: FittedModel, fvs: FVs, method="transform"):
    method_func = getattr(fitted_model, method)
    return method_func(list(fvs))


# ---------------------------------------------------------------------------------------
# The stores that will be used -- here, all stores are just dictionaries, but the
# contract is with the typing.Mapping (read-only here) interface.
# As we grow up, we'll use other mappings, such as:
# - server side RAM (as done here, simply)
# - server side persistence (files or any DB or file system thanks to the dol package)
# - computation (when you want the request for a key to actually launch a process that
#   will generate the value for you (some say you should be obvious to that detail))
# - client side RAM (when we figure that out)

mall = dict(
    fvs=dict(
        train_fvs_1=np.array([[1], [2], [3], [5], [4], [2], [1], [4], [3]]),
        train_fvs_2=np.array([[1], [10], [5], [3], [4]]),
        test_fvs=np.array([[1], [5], [3], [10], [-5]]),
    ),
    fitted_model=dict(
        fitted_model_1=MinMaxScaler().fit(
            [[1], [2], [3], [5], [4], [2], [1], [4], [3]]
        ),
        fitted_model_2=MinMaxScaler().fit([[1], [10], [5], [3], [4]]),
    ),
    model_results=dict(),
)

# ---------------------------------------------------------------------------------------
# dispatchable function:

from streamlitfront.examples.crude.crude_util import auto_key

from i2 import Sig
from i2.wrapper import Ingress, wrap
from inspect import Parameter


def prepare_for_crude_dispatch(func, store_for_param=None, output_store_name=None):
    """Wrap func into something that is ready for CRUDE dispatch."""

    ingress = None
    if store_for_param is not None:
        sig = Sig(func)
        print('sig:', sig)
        crude_params = [x for x in sig.names if x in store_for_param]
        print(f'crude params: {crude_params}')
        def kwargs_trans(outer_kw):
            def gen():
                for store_name in crude_params:
                    store_key = outer_kw[store_name]
                    yield store_name, store_for_param[store_name][store_key]

            return dict(gen())

        save_name_param = Parameter(
            name="save_name",
            kind=Parameter.KEYWORD_ONLY,
            default="",
            annotation=str,
        )
        print(save_name_param)
        outer_sig=(
                sig.ch_annotations(**{name: str for name in crude_params})
                + [save_name_param]
            )
        print(outer_sig)
        ingress = Ingress(
            inner_sig=sig,
            kwargs_trans=kwargs_trans,
            outer_sig=(
                sig.ch_annotations(**{name: str for name in crude_params})
                + [save_name_param]
            ),
        )

        egress = None
        if output_store_name:

            def egress(func_output):
                print(f"{list(store_for_param[output_store_name])=}")
                store_for_param[output_store_name] = func_output
                print(f"{list(store_for_param[output_store_name])=}")
                return func_output

    wrapped_f = wrap(func, ingress, egress)

    return wrapped_f


def prepare_for_crude_dispatch2(func, store_for_param=None, output_store_name=None):
    """Wrap func into something that is ready for CRUDE dispatch."""

    ingress = None
    if store_for_param is not None:
        sig = Sig(func)
        crude_params = [x for x in sig.names if x in store_for_param]

        def kwargs_trans(outer_kw):
            def gen():
                for store_name in crude_params:
                    store_key = outer_kw[store_name]
                    yield store_name, store_for_param[store_name][store_key]

            return dict(gen())

        save_name_param = Parameter(
            name="save_name",
            kind=Parameter.KEYWORD_ONLY,
            default="",
            annotation=str,
        )
        outer_sig=(
                sig.ch_annotations(**{name: str for name in crude_params})
                + [save_name_param]
            )
        print(outer_sig)
        ingress = Ingress(
            inner_sig=sig,
            kwargs_trans=kwargs_trans,
            outer_sig=(
                sig.ch_annotations(**{name: str for name in crude_params})
                + [save_name_param]
            ),
        )

        egress = None
        if output_store_name:

            def egress(func_output):
                print(f"{list(store_for_param[output_store_name])=}")
                store_for_param[output_store_name] = func_output
                print(f"{list(store_for_param[output_store_name])=}")
                return func_output

    wrapped_f = wrap(func, ingress, egress)

    return wrapped_f






f = prepare_for_crude_dispatch(apply_model, mall)
assert all(
    f("fitted_model_1", "test_fvs") == np.array([[0.0], [1.0], [0.5], [2.25], [-1.5]])
)

sig: (fitted_model: Any, fvs: Any, method='transform')
crude params: ['fitted_model', 'fvs']
save_name: str = ''
(fitted_model: str, fvs: str, method='transform', save_name: str = '')


In [8]:
from dataclasses import make_dataclass
from pydantic import BaseModel
CC = make_dataclass('Y', fields=[('s', str), ('x', int)], bases=(BaseModel,))

In [10]:
CC.__fields__

{'s': ModelField(name='s', type=str, required=True),
 'x': ModelField(name='x', type=int, required=True)}

In [11]:
issubclass(CC, BaseModel)

True

In [17]:
func = apply_model
sig = Sig(func)
sig.names

['fitted_model', 'fvs', 'method']

In [16]:
dict(sig)

{'fitted_model': <Parameter "fitted_model: Any">,
 'fvs': <Parameter "fvs: Any">,
 'method': <Parameter "method='transform'">}

In [18]:
from inspect import signature

In [20]:
sg = signature(func)

In [29]:
sg.parameters['fvs'].annotation


typing.Any

In [34]:
sig.annotations.items()

dict_items([('fitted_model', typing.Any), ('fvs', typing.Any)])

In [36]:
from typing import List

mall = {'x':{'xs_1':3, 'xs_2':5}, 'fvs':{'fv_1':[1,2,3], 'fv_2':[4,5,6]}}


def f(x:int, fvs:List[int]):
    return [x*item for item in fvs]

sig_list = Sig(f).annotations.items()
CC = make_dataclass('Input', fields=sig_list, bases=(BaseModel,))

In [50]:
import streamlit_pydantic as sp
import streamlit as st
from pydantic import BaseModel, Field, ValidationError, parse_obj_as

class InputFromStore:
    pass
from enum import Enum
from dataclasses import field

CC = make_dataclass('Input', fields=[('FOO',str, field(init='foo')),('BAR', str, field(init='bar'))], bases=(str,Enum,))

class SelectionValue(str, Enum):
    FOO = "foo"
    BAR = "bar"


def mk(store, key):
    choices = store[key].keys()

class ExampleModel(BaseModel):
    single_selection: SelectionValue= Field(
        ..., description="Only select a single item from a set."
    )
    


data = sp.pydantic_form(key="my_form", model=ExampleModel)
if data:
    st.json(data.json())


2022-02-04 15:45:25.091 WARNING streamlit.state.session_state: Session state does not function when running a script without `streamlit run`


KeyError: 'st.session_state has no key "my_form-data". Did you forget to initialize it? More info: https://docs.streamlit.io/library/advanced-features/session-state#initialization'

In [43]:
CC.__annotations__

{'FOO': str, 'BAR': str}

In [51]:
Input = make_dataclass(
    "Input",
    fields=[("FOO", str, field(init="foo")), ("BAR", str, field(init="bar"))],
    bases=(
        str,
        Enum,
    ),
)

In [57]:
dir(Input)

['__class__', '__doc__', '__members__', '__module__']

In [66]:
from dataclasses import dataclass
make_dataclass('Input',fields=[('x',int)])

types.Input

In [73]:

mall = {"x": {"xs_1": 3, "xs_2": 5}, "fvs": {"fv_1": [1, 2, 3], "fv_2": [4, 5, 6]}}


def mk_Enum_from_store(store, key):
    choices = store[key].keys()
    choice = Enum(f"DynamicEnum_{key}", {item: item for item in choices})
    return choice


class ExampleModel2(BaseModel):
    fvs: mk_Enum_from_store(mall, key="x")



In [74]:
from pydantic import BaseModel, create_model

DynamicFoobarModel = create_model('ExampleModel3', fvs=(mk_Enum_from_store(mall, key="x"), ...))




In [78]:
from typing_extensions import TypedDict
Movie = TypedDict('Movie', {'name': str, 'year': int})

In [79]:
from pydantic import create_model_from_typeddict
UserM = create_model_from_typeddict(Movie)


In [80]:
UserM

pydantic.main.Movie

In [81]:
from dol import Files


ImportError: cannot import name 'Files' from 'dol' (/Users/sylvain/opt/anaconda3/envs/otopy38/lib/python3.8/site-packages/dol/__init__.py)

In [1]:
from recode import decode_wav_bytes

In [4]:
decode_wav_bytes??

Signature: decode_wav_bytes(wav_bytes: bytes)
Source:   
def decode_wav_bytes(wav_bytes: bytes):
    r"""

    :param width: The width of a sample (in bits, bytes, numpy dtype, pyaudio ...)
        (Will try to figure it out)
    :param n_channels: Number of channels
    :return: The decoded waveform

    >>> wav_bytes = (
    ...     b'RIFF.\x00\x00\x00WAVEfmt \x10\x00\x00\x00\x01\x00\x01\x00'  # header
    ...     b'*\x00\x00\x00T\x00\x00\x00\x02\x00\x10\x00data\n\x00\x00\x00'  # header
    ...     b'\x00\x00\x01\x00\xff\xff\x02\x00\xfe\xff'  # data
    ... )
    >>> wf, sr = decode_wav_bytes(wav_bytes)
    >>> wf
    [0, 1, -1, 2, -2]
    >>> sr
    42
    """
    meta = decode_wav_header_bytes(wav_bytes)
    header_size = header_size_of_wav_bytes(wav_bytes, meta)
    wf = decode_pcm_bytes(
        wav_bytes[header_size:],
        width=meta['width_bytes'],
        n_channels=meta['n_channels'],
    )
    return wf, meta['sr']
File:      ~/Desktop/dev/otosense/recode/recode/audio.py

In [5]:
path = '/Users/sylvain/Dropbox/Otosense/VacuumEdgeImpulse/train/noise.Babble_1.2.320000-480000.wav.23q8e2h3.ingestion-6bc8b65f8c-vrv59.wav'
with open(path, 'rb') as fp:
    b = fp.read()
    wf, sr = decode_wav_bytes(b)


In [1]:
from ptr.evacuation.stores import Dacc

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
dacc = Dacc()

In [4]:
dacc.wfs

FilesOfZip('None', '', {}, <function ZipReader.FILES_ONLY at 0x7fccbc1fa940>)

In [6]:
list(dacc.wfs.keys())[:10]

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [9]:
dacc.annots_df

,filename,tag,key,group
0,Session_001.wav,discard,1,0
1,Session_002.wav,discard,2,0
2,Session_003.wav,discard,3,0
3,Session_004.wav,discard,4,0
4,Session_005.wav,discard,5,0
5,Session_006.wav,discard,6,0
6,Session_007.wav,closed,7,1
7,Session_008.wav,closed,8,1
8,Session_009.wav,closed,9,1
9,Session_010.wav,closed,10,1
